In [189]:
import pandas as pd
from datetime import datetime, timedelta
import re
from collections import Counter

In [3]:
df = pd.read_csv('metadata_cut_clean.csv')

In [4]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...,NaN
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,y2nhss9u,62b7f419bb3a4af2f439778b85c0b2582461053a,PMC,Nucleolus: the fascinating nuclear body,10.1007/s00418-007-0359-6,PMC2137947,18046571.0,no-cc,Nucleoli are the prominent contrasted structur...,2007-11-29,"Sirri, Valentina; Urcuqui-Inchima, Silvio; Rou...",Histochem Cell Biol,NaN,NaN,NaN,document_parses/pdf_json/62b7f419bb3a4af2f4397...,document_parses/pmc_json/PMC2137947.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN
4,zurd5d64,66f425e847bc112e32f81cac4c2c57b1b6d0f284,PMC,Autoimmune Cholangitis in the SJL/J Mouse is A...,10.1080/1044667021000096455,PMC2276095,12739787.0,cc-by,Primary biliary cirrhosis (PBC) is an autoimmu...,2002-06-16,"Sasaki, Motoko; Allina, Jorge; Odin, Joseph A....",Dev Immunol,NaN,NaN,NaN,document_parses/pdf_json/66f425e847bc112e32f81...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9318 entries, 0 to 9317
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cord_uid          9318 non-null   object 
 1   sha               8091 non-null   object 
 2   source_x          9318 non-null   object 
 3   title             9318 non-null   object 
 4   doi               8954 non-null   object 
 5   pmcid             6960 non-null   object 
 6   pubmed_id         6481 non-null   float64
 7   license           9318 non-null   object 
 8   abstract          9318 non-null   object 
 9   publish_time      8804 non-null   object 
 10  authors           9169 non-null   object 
 11  journal           7806 non-null   object 
 12  mag_id            0 non-null      float64
 13  who_covidence_id  380 non-null    object 
 14  arxiv_id          257 non-null    object 
 15  pdf_json_files    8091 non-null   object 
 16  pmc_json_files    6041 non-null   object 


In [6]:
def ensure_datetime_format(df, column_name='publish_time'):
    # Check if the column is already in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df[column_name]):
        # Convert to datetime, coercing invalid dates to NaT
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    
    return df

In [7]:
ensure_datetime_format(df, 'publish_time')

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...,NaN
1,1n0rg5vd,6af02873a430f5f75dc56a917f26feaea89b8e79,PMC,The Effectiveness of Contact Tracing in Emergi...,10.1371/journal.pone.0000012,PMC1762362,17183638.0,cc-by,BACKGROUND: Contact tracing plays an important...,2006-12-20,"Klinkenberg, Don; Fraser, Christophe; Heesterb...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/6af02873a430f5f75dc56...,document_parses/pmc_json/PMC1762362.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
2,nzh87aux,9b7a0ad7b6c7f59e7a6cf1dc9d07912a273d19b5,PMC,The Waiting Time for Inter-Country Spread of P...,10.1371/journal.pone.0000143,PMC1764036,17206278.0,cc-by,BACKGROUND: The time delay between the start o...,2007-01-03,"Caley, Peter; Becker, Niels G.; Philp, David J.",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/9b7a0ad7b6c7f59e7a6cf...,document_parses/pmc_json/PMC1764036.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
3,y2nhss9u,62b7f419bb3a4af2f439778b85c0b2582461053a,PMC,Nucleolus: the fascinating nuclear body,10.1007/s00418-007-0359-6,PMC2137947,18046571.0,no-cc,Nucleoli are the prominent contrasted structur...,2007-11-29,"Sirri, Valentina; Urcuqui-Inchima, Silvio; Rou...",Histochem Cell Biol,NaN,NaN,NaN,document_parses/pdf_json/62b7f419bb3a4af2f4397...,document_parses/pmc_json/PMC2137947.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN
4,zurd5d64,66f425e847bc112e32f81cac4c2c57b1b6d0f284,PMC,Autoimmune Cholangitis in the SJL/J Mouse is A...,10.1080/1044667021000096455,PMC2276095,12739787.0,cc-by,Primary biliary cirrhosis (PBC) is an autoimmu...,2002-06-16,"Sasaki, Motoko; Allina, Jorge; Odin, Joseph A....",Dev Immunol,NaN,NaN,NaN,document_parses/pdf_json/66f425e847bc112e32f81...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9313,15rcq4sh,ab10fed7d8688da4c396efac79aa05d8d6b8f300,Medline; PMC,Immunofluorescence Assay for Serologic Diagnos...,10.3201/eid1003.030493,PMC3322771,15109430.0,no-cc,We evaluated a virus-infected cell-based indir...,2004-03-16,"Chan, Paul K. S.; Ng, King-Cheung; Chan, Rickj...",Emerg Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/ab10fed7d8688da4c396e...,document_parses/pmc_json/PMC3322771.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/15109430/,7366198.0
9314,dg213uf6,771674c2114b18c2aed942a6d1da598a3009652d,Medline; PMC,Response to the COVID-19 Epidemic: The Chinese...,10.3390/ijerph17072304,PMC7177503,32235413.0,cc-by,The ongoing outbreak of the novel coronavirus ...,2020-03-30,"Liu, Wei; Yue, Xiao-Guang; Tchounwou, Paul B.",Int J Environ Res Public Health,NaN,NaN,NaN,document_parses/pdf_json/771674c2114b18c2aed94...,document_parses/pmc_json/PMC7177503.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32235413/;...,214768882.0
9315,enhx4abh,ce8106b79db566c503b0343190cf18a399abeda7,Medline; PMC,Respiratory Infections in Travelers Returning ...,10.1007/5584_2014_89,PMC7120182,25381557.0,no-cc,"Respiratory tract infections (RTIs), beside di...",2014-11-09,"Korzeniewski, Krzysztof; Nitsch-Osuch, Aneta; ...",Environmental Biomedicine,NaN,NaN,NaN,document_parses/pdf_json/ce8106b79db566c503b03...,document_parses/pmc_json/PMC7120182.xml.json,https://doi.org/10.1007/5584_2014_89; https://...,24095752.0
9316,k0bf8lnf,728157c34f07e1c21dbe725b96bbb08a9449504c,Medline; PMC,Aerosol-Transmitted Infectionsa New Considerat...,10.1007/s40506-015-0057-1

In [27]:
import requests

In [29]:
# Define the function to fetch citation count for a single DOI
def fetch_citation_count(doi):
    base_url = "https://api.crossref.org/works/"
    try:
        # Query the CrossRef API for the DOI
        response = requests.get(f"{base_url}{doi}")
        response.raise_for_status()
        data = response.json()
        # Extract the citation count (example field: is-referenced-by-count)
        return data["message"].get("is-referenced-by-count", "N/A")
    except Exception as e:
        # Handle errors gracefully
        return f"Error: {e}"

In [32]:
df['referenced_by_count'] = df['doi'].apply(fetch_citation_count)

In [36]:
# Define the function to fetch the journal name for a single DOI
def fetch_journal_name(doi):
    base_url = "https://api.crossref.org/works/"
    try:
        # Query the CrossRef API for the DOI
        response = requests.get(f"{base_url}{doi}")
        response.raise_for_status()
        data = response.json()
        # Extract the journal name (example field: container-title)
        journal_name = data["message"].get("container-title", ["N/A"])
        # Since 'container-title' is a list, return the first entry if it exists
        return journal_name[0] if journal_name else "N/A"
    except Exception as e:
        # Handle errors gracefully
        return f"Error: {e}"

In [37]:
df['JournalName_DOI'] = df['doi'].apply(fetch_journal_name)

In [48]:
pd.set_option('display.max_colwidth', None)

In [49]:
df[['journal', 'JournalName_DOI']]

,journal,JournalName_DOI
1805,NaN,N/A
2857,NaN,N/A
1611,NaN,N/A
2824,NaN,N/A
5302,J Formos Med Assoc,Journal of the Formosan Medical Association
...,...,...
6851,ACS Infectious Diseases,ACS Infectious Diseases
8435,Clin Infect Dis,Clinical Infectious Diseases
8436,Eur J Clin Invest,European Journal of Clinical Investigation
7920,BMJ Open,BMJ Open


In [53]:
df['referenced_by_count'] = pd.to_numeric(df['referenced_by_count'], errors='coerce').fillna(0).astype(int)

In [203]:
def assign_research_tags(row):
    tags = []
    title = str(row['title']).lower()  # Convert to lowercase for case-insensitive matching
    #cited_count = int(row['referenced_by_count']) if pd.notna(row['referenced_by_count']) else 0
    
    # Literature Review
    lit_review_keywords = [
        'review', 'meta-analysis', 'survey', 'overview', 'systematic review', 'literature review', 'comprehensive review', 'critical review', 'scoping review', 'narrative review']
    if any(keyword in title for keyword in lit_review_keywords):
        tags.append('Literature Review')
    
    # Methodologies or Experimental Designs
    method_keywords = [
        'method', 'methodology', 'design', 'experiment', 'approach', 'technique', 'protocol', 'procedure', 'framework', 'model', 'strategy', 'system', 'algorithm', 'paradigm', 'tool', 'instrument', 'guide', 'feasibility of']
    if any(keyword in title for keyword in method_keywords):
        tags.append('Methodologies or Experimental Designs')
    
    # 4. Theoretical Frameworks or Models
    theory_keywords = [
        'theory', 'model', 'framework', 'conceptual', 'theoretical', 'paradigm', 'hypothesis', 'perspective', 'approach', 'lens', 'construct', 'schema', 'ontology', 'epistemology', 'role of']
    if any(keyword in title for keyword in theory_keywords):
        tags.append('Theoretical Frameworks or Models')
    
    # 5. Research Gaps (no time restriction)
    gap_keywords = [
        'gap', 'future research', 'research agenda', 'unexplored', 'understudied', 'limitation', 'challenge', 'opportunity', 'prospect', 'frontier', 'avenue', 'direction', 'need for', 'call for', 'roadblocks']
    if any(keyword in title for keyword in gap_keywords):
        tags.append('Research Gaps')
    
    # 6. Comparative Works
    compare_keywords = [
        'comparison', 'comparative', 'versus', 'vs.', 'vs', 'alternative', 'contrast', 'differ', 'debate', 'controversy', 'disagreement', 'dispute', 'conflict', 'competing', 'multiple', 'diverse', 'varied', 'relationship between']
    if any(keyword in title for keyword in compare_keywords):
        tags.append('Comparative Works')
    
    # 7. Quantitative Data and Analysis
    quant_keywords = [
        'empirical', 'quantitative', 'statistical', 'analysis', 'data', 'case study', 'case-finding', 'case finding', 'regression', 'correlation', 'experiment', 'survey', 'meta-analysis', 'replication', 'evidence', 'results', 'findings', 'dataset', 'database',
        'survey data', 'forecasting', 'forecast', 'predict', 'estimation', 'estimate', 'estimating', 'case series', 'factors impacting', 'determinants', 'causes of', 'implications', 'outcomes', 'evaluation of', 'clinical trial'
    ]
    if any(keyword in title for keyword in quant_keywords):
        tags.append('Quantitative Data and Analysis')
    
    # 8. Exploring Interdisciplinary Connection
    interdisc_keywords = [
        'interdisciplinary', 'multidisciplinary', 'transdisciplinary', 'cross-disciplinary', 'hybrid', 'fusion', 'integration', 'convergence'
    ]
    if any(keyword in title for keyword in interdisc_keywords):
        tags.append('Exploring Interdisciplinary Connection')
    
    # 9. Influential Articles (based on citation count)
    # if cited_count > 50:  # Arbitrary threshold, can be adjusted
    #     tags.append('Influential Articles')
    
    return tags

In [194]:
df['tags'] = df.apply(assign_research_tags, axis=1)

In [195]:
df['TagCount'] = df['tags'].apply(len)

In [196]:
frequency = df['TagCount'].value_counts()
frequency

TagCount
0    5590
1    2453
2     891
3     356
4      28
Name: count, dtype: int64

In [197]:
def assign_strict_abstract_tags(row, initial_tags_col='tags'):
    # Skip if the article already has tags from the initial pass
    if row[initial_tags_col]:  # If initial tags list is not empty
        return row[initial_tags_col]
    
    tags = []
    abstract = str(row['abstract']).lower()  # Use abstract instead of title
    cited_count = int(row['referenced_by_count']) if pd.notna(row['referenced_by_count']) else 0
    
    # Literature Review (stricter: focus on specific review types)
    lit_review_keywords = ['meta-analysis', 'systematic review', 'literature review', 'comprehensive review', 'critical review', 'scoping review', 'narrative review']
    if any(keyword in abstract for keyword in lit_review_keywords):
        tags.append('Literature Review')
    
    # Methodologies or Experimental Designs (stricter: focus on specific methods)
    method_keywords = ['methodology', 'experiment', 'technique', 'protocol', 'procedure', 'algorithm', 'tool', 'instrument', 'feasibility of', 'propose a method']
    if any(keyword in abstract for keyword in method_keywords):
        tags.append('Methodologies or Experimental Designs')
    
    # Theoretical Frameworks or Models (stricter: focus on conceptual terms)
    theory_keywords = ['theory', 'model', 'framework', 'conceptual', 'theoretical', 'hypothesis', 'ontology', 'epistemology', 'role of']
    if any(keyword in abstract for keyword in theory_keywords):
        tags.append('Theoretical Frameworks or Models')
    
    # Research Gaps (stricter: focus on explicit gap signals)
    gap_keywords = ['gap', 'unexplored', 'understudied', 'limitation', 'challenge', 'frontier', 'need for', 'call for', 'roadblocks', 'little is known', 'more research', 'further research']
    if any(keyword in abstract for keyword in gap_keywords):
        tags.append('Research Gaps')
    
    # Comparative Works (stricter: focus on explicit comparison)
    compare_keywords = ['comparison', 'comparative', 'versus', 'vs.', 'vs', 'contrast', 'debate', 'controversy', 'dispute', 'competing']
    if any(keyword in abstract for keyword in compare_keywords):
        tags.append('Comparative Works')
    
    # Quantitative Data and Analysis (stricter: focus on specific quant terms)
    quant_keywords = ['empirical', 'quantitative', 'statistical', 'regression', 'correlation', 'meta-analysis', 'replication', 'data', 'real-time', 'real time', 'dataset', 'database', 'forecasting', 'forecast', 'predict', 'estimation', 'clinical trial', 'case study']
    if any(keyword in abstract for keyword in quant_keywords):
        tags.append('Quantitative Data and Analysis')
    
    # Exploring Interdisciplinary Connection (stricter: keep specific terms)
    interdisc_keywords = ['interdisciplinary', 'multidisciplinary', 'transdisciplinary', 'cross-disciplinary', 'integration', 'convergence']
    if any(keyword in abstract for keyword in interdisc_keywords):
        tags.append('Exploring Interdisciplinary Connection')

    return tags

In [198]:
df['tags'] = df.apply(assign_strict_abstract_tags, axis=1)

In [199]:
df['TagCount'] = df['tags'].apply(len)

In [200]:
frequency = df['TagCount'].value_counts()
frequency

TagCount
1    4133
0    2433
2    1946
3     714
4      84
5       6
7       1
6       1
Name: count, dtype: int64

In [201]:
all_tags = [tag for tags_list in df['tags'] for tag in tags_list]

tag_frequency = Counter(all_tags)

In [202]:
tag_frequency_df = pd.DataFrame(tag_frequency.items(), columns=['Tag', 'Frequency']).sort_values(by='Frequency', ascending=False)
tag_frequency_df

,Tag,Frequency
3,Quantitative Data and Analysis,3761
0,Methodologies or Experimental Designs,2268
1,Theoretical Frameworks or Models,1880
5,Comparative Works,1066
2,Research Gaps,945
4,Literature Review,487
6,Exploring Interdisciplinary Connection,139


In [225]:
df.to_csv("purposes_added_metadata_v2.csv", index=False)

In [147]:
#five_years_ago = datetime.now() - timedelta(days=5*365)

In [151]:
df.to_csv("purposes_metadata.csv", index=False)

In [187]:
filtered_df.to_csv("quant_tagged_articles.csv", index=False)